In [2]:
import os
import pickle
import numpy as np
from fashion_mnist import mnist_reader
from vectorization import *

In [ ]:
path_feat = "fashion_mnist/features/"
path_dia = "fashion_mnist/pdiagrams/"

### Generate and save Barcodes

In [ ]:
images_train_l, y_train = mnist_reader.load_mnist('fashion_mnist/data/fashion', kind='train')
images_test_l, y_test = mnist_reader.load_mnist('fashion_mnist/data/fashion', kind='t10k')
images_train_u = 255 - images_train_l
images_test_u = 255 - images_test_l

In [ ]:
for i in range(60000):
    dgms = GetCubicalComplexPDs(img=images_train_l[i], img_dim=[28,28])
    np.savetxt(path_dia + "l_d0_"+str(i),dgms[0])
    np.savetxt(path_dia + "l_d1_"+str(i),dgms[1])
    
    dgms = GetCubicalComplexPDs(img=images_train_u[i], img_dim=[28,28])
    np.savetxt(path_dia + "u_d0_"+str(i),dgms[0])
    np.savetxt(path_dia + "u_d1_"+str(i),dgms[1])

In [ ]:
for j in range(10000):
    i = 60000 + j 
    dgms = GetCubicalComplexPDs(img=images_test_l[j], img_dim=[28,28])
    np.savetxt(path_dia + "l_d0_"+str(i),dgms[0])
    np.savetxt(path_dia + "l_d1_"+str(i),dgms[1]) 
    
    dgms = GetCubicalComplexPDs(img=images_test_u[j], img_dim=[28,28])
    np.savetxt(path_dia + "u_d0_"+str(i),dgms[0])
    np.savetxt(path_dia + "u_d1_"+str(i),dgms[1]) 

### Load Barcodes

In [3]:
pdiagrams = dict()

#Barcodes with just one bar are loaded as a 1d-array.
#We force them to be a 2d-array
def safe_load(x):
    pd = np.loadtxt(x)
    if (len(pd.shape)==1) and (pd.shape[0]>0): 
        pd = pd.reshape(1,2)
    return pd

for i in range(70000):
    pdiagrams["pdiag_l_d0_"+str(i)]= safe_load(path_dia + "l_d0_"+str(i))
    pdiagrams["pdiag_l_d1_"+str(i)]= safe_load(path_dia + "l_d1_"+str(i))
    pdiagrams["pdiag_u_d0_"+str(i)]= safe_load(path_dia + "u_d0_"+str(i))
    pdiagrams["pdiag_u_d1_"+str(i)]= safe_load(path_dia + "u_d1_"+str(i))

/tmp/ipykernel_133837/933824092.py:6: UserWarning: loadtxt: Empty input file: "fashion_mnist/pdiagrams/u_d1_13024"
  pd = np.loadtxt(x)
/tmp/ipykernel_133837/933824092.py:6: UserWarning: loadtxt: Empty input file: "fashion_mnist/pdiagrams/u_d1_20447"
  pd = np.loadtxt(x)
/tmp/ipykernel_133837/933824092.py:6: UserWarning: loadtxt: Empty input file: "fashion_mnist/pdiagrams/u_d1_25551"
  pd = np.loadtxt(x)
/tmp/ipykernel_133837/933824092.py:6: UserWarning: loadtxt: Empty input file: "fashion_mnist/pdiagrams/u_d1_26737"
  pd = np.loadtxt(x)
/tmp/ipykernel_133837/933824092.py:6: UserWarning: loadtxt: Empty input file: "fashion_mnist/pdiagrams/u_d1_30061"
  pd = np.loadtxt(x)
/tmp/ipykernel_133837/933824092.py:6: UserWarning: loadtxt: Empty input file: "fashion_mnist/pdiagrams/u_d1_32048"
  pd = np.loadtxt(x)
/tmp/ipykernel_133837/933824092.py:6: UserWarning: loadtxt: Empty input file: "fashion_mnist/pdiagrams/u_d1_39009"
  pd = np.loadtxt(x)
/tmp/ipykernel_133837/933824092.py:6: UserWarnin

### Calculate Features

In [4]:
func_list = [GetPersStats,
             GetPersLifespanFeature,
             GetPersImageFeature,
             GetPersLandscapeFeature,
             GetPersEntropyFeature,
             GetBettiCurveFeature,
             GetCarlssonCoordinatesFeature,
             GetPersSilhouetteFeature,
             GetTopologicalVectorFeature,
             #GetAtolFeature,
             GetComplexPolynomialFeature,
             GetPersLifespanFeature,
             GetPersTropicalCoordinatesFeature
            ]

In [5]:
features_l_d0 = dict()
features_l_d1 = dict()
features_u_d0 = dict()
features_u_d1 = dict()

for func in func_list:
    for i in range(70000):
        features_l_d0[(func.__name__)+'_'+str(i)]=func(pdiagrams["pdiag_l_d0_"+str(i)])
        features_l_d1[(func.__name__)+'_'+str(i)]=func(pdiagrams["pdiag_l_d1_"+str(i)])
        features_u_d0[(func.__name__)+'_'+str(i)]=func(pdiagrams["pdiag_u_d0_"+str(i)])
        features_u_d1[(func.__name__)+'_'+str(i)]=func(pdiagrams["pdiag_u_d1_"+str(i)])

/home/manu/review/review/lib/python3.8/site-packages/sklearn/neighbors/_distance_metric.py:10: FutureWarning: sklearn.neighbors.DistanceMetric has been moved to sklearn.metrics.DistanceMetric in 1.0. This import path will be removed in 1.3
  warnings.warn(
/home/manu/review/review/lib/python3.8/site-packages/gudhi/representations/vector_methods.py:579: RuntimeWarning: overflow encountered in cdouble_scalars
  coeff[j] += ((-1) * roots[i-1] * coeff[j+1])
/home/manu/review/review/lib/python3.8/site-packages/gudhi/representations/vector_methods.py:579: RuntimeWarning: invalid value encountered in cdouble_scalars
  coeff[j] += ((-1) * roots[i-1] * coeff[j+1])


In [6]:
with open('fashionMNIST_l_d0.pkl', 'wb') as f:
  pickle.dump(features_l_d0, f)
with open('fashionMNIST_l_d1.pkl', 'wb') as f:
  pickle.dump(features_l_d1, f)
with open('fashionMNIST_u_d0.pkl', 'wb') as f:
  pickle.dump(features_u_d0, f)
with open('fashionMNIST_u_d1.pkl', 'wb') as f:
  pickle.dump(features_u_d1, f)